In [1]:
GPU_device_id = str(3)
model_id_save_as = 'learningcurve-dnn-easy-final'
architecture_id = 'final-models/learningcurve-dnn-easy-final-features'
model_class_id = 'DNN'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_200keV_log10time_10000.npy'
difficulty_setting = 'easy'

train_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000,]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [4]:
from annsa.model_classes import build_dnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [6]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [7]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [5]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.softmax
model_features.metrics = [f1]
model_features.learning_rate = model_features.learining_rate

In [7]:
model = compile_model(
            build_dnn_model,
            model_features)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                65600     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1950      
Total params: 67,550
Trainable params: 67,550
Non-trainable params: 0
_________________________________________________________________


# Train network

## Scale input data

In [12]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [6]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [14]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        model = compile_model(
            build_dnn_model,
            model_features)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 50
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 2s 40ms/sample - loss: 29.9896 - f1: 0.0590 - val_loss: 28.5813 - val_f1: 0.0323
Epoch 2/2000
50/50 [==============================] - 0s 4ms/sample - loss: 28.5834 - f1: 0.0479 - val_loss: 26.6575 - val_f1: 0.0098
Epoch 3/2000
50/50 [==============================] - 0s 4ms/sample - loss: 26.9324 - f1: 0.0172 - val_loss: 25.7215 - val_f1: 0.0205
Epoch 4/2000
50/50 [==============================] - 0s 4ms/sample - loss: 25.6597 - f1: 0.0800 - val_loss: 24.6484 - val_f1: 0.0277
Epoch 5/2000
50/50 [==============================] - 0s 4ms/sample - loss: 24.5101 - f1: 0.0900 - val_loss: 23.1004 - val_f1: 0.0344
Epoch 6/2000
50/50 [==============================] - 0s 4ms/sample - loss: 23.1695 - f1: 0.1140 - val_loss: 22.1958 - val_f1: 0.0517
Epoch 7/2000
50/50 [==============================] - 0s 4ms/sample - loss: 21.9344 - f1: 0.0697 - val_loss: 20.7921 - val_f1: 0.0366
Epoch 8/2000
50

Epoch 9/2000
50/50 [==============================] - 0s 4ms/sample - loss: 19.5339 - f1: 0.1413 - val_loss: 18.6420 - val_f1: 0.0410
Epoch 10/2000
50/50 [==============================] - 0s 4ms/sample - loss: 18.0855 - f1: 0.0625 - val_loss: 17.4152 - val_f1: 0.0345
Epoch 11/2000
50/50 [==============================] - 0s 4ms/sample - loss: 16.7097 - f1: 0.0893 - val_loss: 16.0043 - val_f1: 0.0315
Epoch 12/2000
50/50 [==============================] - 2s 43ms/sample - loss: 15.2789 - f1: 0.0698 - val_loss: 14.8204 - val_f1: 0.0253
Running through fold 4
Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 2s 45ms/sample - loss: 29.5934 - f1: 0.0159 - val_loss: 28.0202 - val_f1: 0.0151
Epoch 2/2000
50/50 [==============================] - 0s 4ms/sample - loss: 27.7364 - f1: 0.0333 - val_loss: 27.0807 - val_f1: 0.0220
Epoch 3/2000
50/50 [==============================] - 0s 4ms/sample - loss: 26.7684 - f1: 0.0467 - val_loss: 25.8206 - val

100/100 [==============================] - 0s 2ms/sample - loss: 13.7332 - f1: 0.1118 - val_loss: 13.3613 - val_f1: 0.0846
Epoch 10/2000
100/100 [==============================] - 0s 2ms/sample - loss: 12.8439 - f1: 0.0603 - val_loss: 12.6919 - val_f1: 0.0670
Epoch 11/2000
100/100 [==============================] - 0s 2ms/sample - loss: 12.6003 - f1: 0.1524 - val_loss: 11.8818 - val_f1: 0.0737
Epoch 12/2000
100/100 [==============================] - 0s 2ms/sample - loss: 11.2916 - f1: 0.1276 - val_loss: 11.1069 - val_f1: 0.0581
Epoch 13/2000
100/100 [==============================] - 0s 2ms/sample - loss: 10.4322 - f1: 0.0563 - val_loss: 10.2200 - val_f1: 0.0223
Epoch 14/2000
100/100 [==============================] - 0s 2ms/sample - loss: 9.8032 - f1: 0.0147 - val_loss: 9.9712 - val_f1: 0.0069
Epoch 15/2000
100/100 [==============================] - 0s 2ms/sample - loss: 9.5608 - f1: 0.0000e+00 - val_loss: 9.5320 - val_f1: 0.0086
Epoch 16/2000
100/100 [==============================] 

...

10000/10000 [==============================] - 1s 142us/sample - loss: 0.5199 - f1: 0.9030 - val_loss: 0.3876 - val_f1: 0.9387
Epoch 30/2000
10000/10000 [==============================] - 1s 143us/sample - loss: 0.5713 - f1: 0.8784 - val_loss: 0.5084 - val_f1: 0.8919
Epoch 31/2000
10000/10000 [==============================] - 1s 140us/sample - loss: 0.5345 - f1: 0.8915 - val_loss: 0.4071 - val_f1: 0.9280
Epoch 32/2000
10000/10000 [==============================] - 1s 137us/sample - loss: 0.5182 - f1: 0.8988 - val_loss: 0.4018 - val_f1: 0.9291
Epoch 33/2000
10000/10000 [==============================] - 5s 492us/sample - loss: 0.5058 - f1: 0.9046 - val_loss: 0.3827 - val_f1: 0.9436
Running through fold 2
Train on 10000 samples, validate on 3000 samples
Epoch 1/2000
10000/10000 [==============================] - 5s 466us/sample - loss: 7.5665 - f1: 0.1672 - val_loss: 3.0663 - val_f1: 0.3568
Epoch 2/2000
10000/10000 [==============================] - 1s 141us/sample - loss: 2.0775 - f1: 

15000/15000 [==============================] - 2s 133us/sample - loss: 0.5040 - f1: 0.9070 - val_loss: 0.4013 - val_f1: 0.9531
Epoch 15/2000
15000/15000 [==============================] - 2s 134us/sample - loss: 0.5106 - f1: 0.9092 - val_loss: 0.5093 - val_f1: 0.9178
Epoch 16/2000
15000/15000 [==============================] - 2s 133us/sample - loss: 0.5122 - f1: 0.9089 - val_loss: 0.4518 - val_f1: 0.9339
Epoch 17/2000
15000/15000 [==============================] - 2s 133us/sample - loss: 0.4947 - f1: 0.9107 - val_loss: 0.4213 - val_f1: 0.9387
Epoch 18/2000
15000/15000 [==============================] - 2s 134us/sample - loss: 0.4841 - f1: 0.9166 - val_loss: 0.4015 - val_f1: 0.9456
Epoch 19/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.5131 - f1: 0.9041 - val_loss: 0.4452 - val_f1: 0.9262
Epoch 20/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.5190 - f1: 0.9024 - val_loss: 0.4579 - val_f1: 0.9312
Epoch 21/2000
15000/15000 [

15000/15000 [==============================] - 2s 136us/sample - loss: 0.6124 - f1: 0.8739 - val_loss: 0.5794 - val_f1: 0.8778
Epoch 10/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.5853 - f1: 0.8795 - val_loss: 0.4923 - val_f1: 0.9107
Epoch 11/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.6034 - f1: 0.8708 - val_loss: 0.6031 - val_f1: 0.8488
Epoch 12/2000
15000/15000 [==============================] - 2s 135us/sample - loss: 0.5888 - f1: 0.8764 - val_loss: 0.5199 - val_f1: 0.8830
Epoch 13/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.5627 - f1: 0.8868 - val_loss: 0.5751 - val_f1: 0.8657
Epoch 14/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.5752 - f1: 0.8824 - val_loss: 0.4542 - val_f1: 0.9218
Epoch 15/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.5719 - f1: 0.8847 - val_loss: 0.4465 - val_f1: 0.9310
Epoch 16/2000
15000/15000 [

20000/20000 [==============================] - 3s 135us/sample - loss: 0.4725 - f1: 0.9135 - val_loss: 0.4744 - val_f1: 0.9006
Epoch 20/2000
20000/20000 [==============================] - 3s 132us/sample - loss: 0.4628 - f1: 0.9162 - val_loss: 0.5154 - val_f1: 0.8820
Epoch 21/2000
20000/20000 [==============================] - 7s 346us/sample - loss: 0.4677 - f1: 0.9115 - val_loss: 0.3814 - val_f1: 0.9483
Running through fold 2
Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 7s 325us/sample - loss: 4.3667 - f1: 0.4200 - val_loss: 1.5284 - val_f1: 0.6722
Epoch 2/2000
20000/20000 [==============================] - 3s 134us/sample - loss: 1.1379 - f1: 0.7594 - val_loss: 0.8610 - val_f1: 0.8043
Epoch 3/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.8244 - f1: 0.8240 - val_loss: 0.6445 - val_f1: 0.8832
Epoch 4/2000
20000/20000 [==============================] - 3s 134us/sample - loss: 0.7180 - f1: 0.

Epoch 6/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.6209 - f1: 0.8687 - val_loss: 0.5600 - val_f1: 0.8807
Epoch 7/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.5797 - f1: 0.8809 - val_loss: 0.4857 - val_f1: 0.9080
Epoch 8/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.5618 - f1: 0.8822 - val_loss: 0.4564 - val_f1: 0.9281
Epoch 9/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.5418 - f1: 0.8927 - val_loss: 0.4598 - val_f1: 0.9228
Epoch 10/2000
20000/20000 [==============================] - 3s 134us/sample - loss: 0.5338 - f1: 0.8922 - val_loss: 0.4590 - val_f1: 0.9145
Epoch 11/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.5525 - f1: 0.8853 - val_loss: 0.5370 - val_f1: 0.8803
Epoch 12/2000
20000/20000 [==============================] - 3s 136us/sample - loss: 0.4883 - f1: 0.9078 - val_loss: 0.4447 - val_f1: 0.9145
Epoch 13/2000
200